In [ ]:
from __future__ import print_function
from astropy.io import fits
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from numpy.polynomial.polynomial import polyval, polyder
%matplotlib inline

In [ ]:
from AlgorithmDev import PolygonClipping

In [ ]:
def plot_three_fits_trace(spectral1, spectral_flat, spectral_ref, total_rows, coeffs_rows):
    plt.figure(figsize=(20,20))
    plt.subplot(1, 3, 1) 
    im1 = plt.imshow(spectral1['data'], cmap='gray', norm=LogNorm())
    
    x_val = list(range(0, spectral['xdim']))
    
    #for y in range(0, total_rows):
    #    y_val = [polyval(x, coeffs_rows[y]) for x in x_val]
    #    plt.plot(x_val, y_val, 'r--')
    
    plt.ylim(0, spectral1['ydim'])
    plt.colorbar(im1, fraction=0.046, pad=0.04)   
    
    plt.subplot(1, 3, 2)
    im2 = plt.imshow(spectral_flat['data'], cmap='gray', norm=LogNorm())
    
    plt.ylim(0, spectral_flat['ydim'])
    plt.colorbar(im2, fraction=0.046, pad=0.04)  
    
    plt.subplot(1, 3, 3)
    im2 = plt.imshow(spectral_ref['data'], cmap='gray', norm=LogNorm())
    
    plt.ylim(0, spectral_ref['ydim'])
    plt.colorbar(im2, fraction=0.046, pad=0.04)  

In [ ]:
def plot_output(optimal_data, sum_data, ref_data, order, width):
    # show output
    size = np.shape(optimal_data)
    x_val = list(range(0, size[0]))
    
    #ratio = [ref_data[x]/optimal_data[x] for x in x_val]
    #nzero = list(filter(lambda x: x > 1.7 and x < 2.3, ratio))
    #print('ratio: ', ratio)
    #print('nzero: ', nzero)
    #r = sum(nzero)/len(nzero)    
    r = 2
    #print('average ratio', r)
    
    plt.figure(figsize=(12, 12))
    plt.subplot(1, 1, 1)
    plt.plot(x_val, optimal_data, '-r', label='optimal')
    #plt.plot(x_val, sum_data, '-b', label='sum')
    plt.plot(x_val, ref_data, '-k', label='reference')
    plt.plot(x_val, optimal_data*r, '-m', label='optimal*2')
    
    #plt.plot(x_val, [ratio[x]*1000 for x in x_val], '-g', label='ratio')
    
    plt.title("extraction on order = "+str(order)+", width = "+str(width))
    plt.legend(loc='upper right', prop={'size': 16})
    #plt.colorbar(im, fraction=0.046, pad=0.04)

In [ ]:
def plot_trace_band(trace_data, x1, x2, y1, y2, title=''):  
    display_color = ['-r', '-b', '-k', '-g', '-y', '-m', '-c', '-.r', '-.b', '-.k', '-.g', '-.y', '-.m', '-.c']
    
    plt.figure(figsize=(12, 12))
    total_plot = y2 - y1 + 1
    x_val = list(range(x1, x2))   
    plt.subplot(1, 1, 1)
    
    for y in range(y1, y2+1):
        idx = (y-y1)%(len(display_color))
        plt.plot(x_val, trace_data[y, x1:x2], display_color[idx], label='y = '+str(y))
    
    plt.title(title)
    plt.legend(loc='upper right', prop={'size': 12})
        

In [ ]:
def load_spectral_sample(fits_file, order_trace_csv, flatlamp_file, ref_fits, width):
    poly_c = PolygonClipping(fits_file, width)
    spectral = poly_c.load_paras_spectral()
    print('data size: ', spectral['xdim'], spectral['ydim'])
    coeffs_rows = poly_c.load_csv_file(order_trace_csv)
    flatlamp_spectral = poly_c.load_paras_spectral(flatlamp_file)
    ref_spectral = poly_c.load_paras_spectral(ref_fits)
   
    return {'spectral': spectral, 'flatlamp_spectral': flatlamp_spectral, 'coeffs': coeffs_rows, 
            'ref_spectral': ref_spectral, 'poly_handle': poly_c}

In [ ]:
def analyze_spectral(spectral, f_spectral, coeffs_rows, poly_c, row_idx):
    total_rows = len(coeffs_rows)
    in_data = np.array(spectral.get('data'), None)
    flat_data = np.array(f_spectral.get('data'), None)
    out_data = np.zeros((total_rows, spectral.get('xdim')))
        
    print("optimal extraction on row: ", row_idx)
    optimal_data = poly_c.rectify_spectral_curve_by_optimal2(coeffs_rows[row_idx], in_data, flat_data, verbose=False)
    o_line_data = optimal_data.get('out_data')

    print('sum extraction on row: ', row_idx)
    sum_data = poly_c.rectify_spectral_curve_by_sum(coeffs_rows[row_idx], in_data)
          
    print('done')
    return {'optimal_data': o_line_data, 'sum_data': sum_data, 
            'rectified_trace': optimal_data.get('rectified_trace')} 
    

In [ ]:
def make_fits(data, output_fits):
    hdu = fits.PrimaryHDU(data)
    hdu.writeto(output_fits)

In [ ]:
def spectral_update(spectrall, correct_file):
    in_data = np.array(spectral.get('data'), None)
    
    plt.figure(figsize=(16, 16))
    plt.subplot(1, 1, 1)
    im = plt.imshow(in_data, cmap='gray', norm=LogNorm())
    plt.ylim(0, spectrall['ydim'])
    plt.title("a0018.fits")
    plt.colorbar(im, fraction=0.046, pad=0.04)   
    
    
    correct_data = poly_c.load_correct_data(correct_file)
    new_data = poly_c.correct_data_by_sub(correct_data, spectrall.get('data'))
    spectrall.update({'data': new_data})

    plt.figure(figsize=(16, 16))
    plt.subplot(1, 1, 1)
    im = plt.imshow(correct_data, cmap='gray', norm=LogNorm())
    plt.ylim(0, spectrall['ydim'])
    plt.title("bleeding_cure_14feb2015_1800.fits")
    plt.colorbar(im, fraction=0.046, pad=0.04)  

    plt.figure(figsize=(16, 16))
    plt.subplot(1, 1, 1)
    im = plt.imshow(spectrall.get('data'), cmap='gray', norm=LogNorm())
    plt.ylim(0, spectrall['ydim']) 
    plt.title("after subtraction")
    plt.colorbar(im, fraction=0.046, pad=0.04)   

In [ ]:
fits_base = '../tmp/test_data/paras_data/14feb2015/a00'
csv_base = '../tmp/test_data/paras_data/order_trace_'
output_base = '../tmp/test_data/paras_data/results_optimal/result_'
flats_base = '../tmp/test_data/paras_data/paras.flat'
fits_list = ['18', '19']
fiber_list = ['A', 'B']


In [ ]:
f = 0
c = 0

fiber_name = fiber_list[c]
fits_name = fits_list[f]
width = 6

ref_fits = '../tmp/test_data/paras_data/14feb2015/a00' + fits_name + '_e2ds_' + fiber_name + '.fits'
fits_file = fits_base+fits_name+'.fits'
csv_file = csv_base+fiber_name+'.csv'
flatlamp_file = flats_base+fiber_name+'.fits'

output_fits = output_base + fiber_name + '_' + fits_name + '_optimal.fits'
print('output file: ', output_fits)
sample_info = load_spectral_sample(fits_file, csv_file, flatlamp_file, ref_fits, width)

spectral = sample_info.get('spectral')
flatlamp_spectral = sample_info.get('flatlamp_spectral')
ref_spectral = sample_info.get('ref_spectral')
coeffs_rows = sample_info.get('coeffs')
poly_c = sample_info.get('poly_handle')
plot_three_fits_trace(spectral, flatlamp_spectral, ref_spectral, len(coeffs_rows), coeffs_rows)
order = 30

## Optimal Extraction without subtracting the correction data:

In [ ]:
"""
result_data = analyze_spectral(spectral, flatlamp_spectral, coeffs_rows, poly_c, order)
o_data = result_data.get('optimal_data')[0, ]
s_data = result_data.get('sum_data')[0, ]
rectified_trace = result_data.get('rectified_trace')
#import pdb; pdb.set_trace()
s = np.shape(rectified_trace)
plot_trace_band(rectified_trace, 0, s[1]-1, 0, s[0]-1, 'rectified trace on order '+str(order))
ref_data = ref_spectral.get('data')
plot_output(o_data, s_data, ref_data[order, ], order, width)
"""

## Optimal extraction by subtracting the correction data: 

In [ ]:
spectral_update(spectral, "../tmp/test_data/paras_data/14feb2015/bleeding_cure_14feb2015_1800.fits")

In [ ]:
result_data = analyze_spectral(spectral, flatlamp_spectral, coeffs_rows, poly_c, order)
o_data = result_data.get('optimal_data')[0, ]
s_data = result_data.get('sum_data')[0, ]
rectified_trace = result_data.get('rectified_trace')
#import pdb; pdb.set_trace()
s = np.shape(rectified_trace)
plot_trace_band(rectified_trace, 0, s[1]-1, 0, s[0]-1, 'rectified trace on order '+str(order))
ref_data = ref_spectral.get('data')
plot_output(o_data, s_data, ref_data[order, ], order, width)